In [1]:
import pandas as pd
import os
import collections
import csv
import logging
import numpy as np
import datetime as datetime
import re
import pickle
from pprint import pprint
from tqdm import tqdm
import os

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
%matplotlib inline

# from tensorflow.keras.layers import Input
# import tensorflow as tf
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input, Dense, Reshape, Embedding, Concatenate, dot
# from tensorflow.keras.preprocessing.sequence import skipgrams
# from tensorflow.keras.preprocessing import sequence
# from tensorflow.keras.losses import cosine_similarity
# from tensorflow.keras.callbacks import TensorBoard
# from tensorboard.plugins import projector

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [4]:
!which jupyter

/home/ubuntu/thesis_env2/bin/jupyter


In [5]:
df = pd.read_pickle('./data/df_processed_bigrams.pickle')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 365200 entries, 0 to 369046
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   author             181507 non-null  object        
 1   date               365200 non-null  datetime64[ns]
 2   domain             365200 non-null  object        
 3   title              365115 non-null  object        
 4   url                365200 non-null  object        
 5   content            365200 non-null  object        
 6   topic_area         365200 non-null  object        
 7   content_processed  365200 non-null  object        
dtypes: datetime64[ns](1), object(7)
memory usage: 25.1+ MB


In [7]:
df.head(1)

,author,date,domain,title,url,content,topic_area,content_processed
0,Thomas Hughes,2020-01-02,marketbeat,Three Industrial Giants You Should Own In 2020,https://www.marketbeat.com/originals/three-ind...,With the end of the year just around the corne...,business,end year corner past time think positioning fo...


In [8]:
# Note to do - need to add time element

def log_newline(self, how_many_lines=1):
    file_handler = None
    if self.handlers:
        file_handler = self.handlers[0]

    # Switch formatter, output a blank line
    file_handler.setFormatter(self.blank_formatter)
    for i in range(how_many_lines):
        self.info('')

    # Switch back
    file_handler.setFormatter(self.default_formatter)

def logger_w2v():
    
    log_file = os.path.join('./data', 'word2vec.log')
    print('log file location: ', log_file)
    
    log_format= '%(asctime)s - %(levelname)s - [%(module)s]\t%(message)s'
    formatter = logging.Formatter(fmt=(log_format))
    
    fhandler = logging.FileHandler(log_file)
    fhandler.setFormatter(formatter)
    
    logger = logging.getLogger('word2vec')
    logger.setLevel(logging.DEBUG)
    logger.addHandler(fhandler)
    logger.default_formatter = formatter
    logger.blank_formatter = logging.Formatter(fmt="")
    logger.newline = types.MethodType(log_newline, logger)
    
    return logger
    

# LDA

https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/  
https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24

LDA considers each document as a collection of topics in a certain proportion. And each topic as a collection of keywords, again, in a certain proportion.

Once you provide the algorithm with the number of topics, all it does it to rearrange the topics distribution within the documents and keywords distribution within the topics to obtain a good composition of topic-keywords distribution.

In [9]:
def tokenise_dataset(df):

    tokens = df['content_processed'].str.split(" ")

    return tokens

In [10]:
words = tokenise_dataset(df)
print(len(words))
sorted(words[4][:10])

365200


['carpets',
 'celebrities',
 'coronavirus',
 'life',
 'normal',
 'pandemic',
 'premieres',
 'red',
 'returns',
 'walk']

In [11]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [12]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

lemm = False

# Do lemmatization keeping only noun, adj, vb, adv
# ~ 2 hours to run
if lemm:
    data_lemmatized = lemmatization(words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
    with open('data/data_lemmatized.pickle', 'wb') as f:
        pickle.dump(data_lemmatized, f, pickle.HIGHEST_PROTOCOL)
else:
    with open('data/data_lemmatized.pickle', 'rb') as f:
        data_lemmatized = pickle.load(f)

print(type(data_lemmatized))
print(len(data_lemmatized))
print(len(data_lemmatized[:1][0]))
print(len(words[0]))

<class 'list'>
365200
507
528


In [13]:
# Create dictionary
dict_top_mod = corpora.Dictionary(data_lemmatized)

print(dict_top_mod)
print(len(dict_top_mod))

# check given words by index
print(dict_top_mod[0], dict_top_mod[10], dict_top_mod[20], dict_top_mod[100], dict_top_mod[101], dict_top_mod[102])

Dictionary(876229 unique tokens: ['2nd', 'addition', 'alltime_high', 'america', 'analyst']...)
876229
2nd ba boeing feed financial find


In [14]:
# Filter out terms
no_below = 25 # Infrequent words: minimum number of documents term must appear in
no_above = 0.3 # Frequent words: remove tokens that appear in this % of documents
keep_n = 100000 # Keep only this many words in dictionary
dict_top_mod.filter_extremes(no_below=no_below, no_above=no_above, keep_n=keep_n)

print(len(dict_top_mod))

# check given words by index
print(dict_top_mod[0], dict_top_mod[10], dict_top_mod[20], dict_top_mod[100], dict_top_mod[101], dict_top_mod[102])
print(dict_top_mod)

64310
2nd ba boeing generally get globally
Dictionary(64310 unique tokens: ['2nd', 'addition', 'alltime_high', 'america', 'analyst']...)


In [15]:
# Term document frequency - creates tuples of word frequences (word idx, word_freq), e.g. [[(0,1), (1,4)]]: word at index 0 occurs once in doc, word at index 1 occurs 4 times in doc
corpus_bow = [dict_top_mod.doc2bow(word) for word in data_lemmatized]

print(len(corpus_bow))
print(len(corpus_bow[0]))

# Human readable format of corpus (term-frequency)
[[(dict_top_mod[idx], freq) for idx, freq in cp] for cp in corpus_bow[:1]][0][:10]

365200
266


[('2nd', 1),
 ('addition', 1),
 ('alltime_high', 1),
 ('america', 1),
 ('analyst', 2),
 ('announce', 1),
 ('anti', 1),
 ('aristocrat', 2),
 ('attractive', 1),
 ('august', 1)]

In [24]:
# Build LDA model - BOW
# Started 9:55am still finished `between 12:00 and 1:50
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus_bow[:50000],
                                           id2word=dict_top_mod,
                                           num_topics=250, 
                                           random_state=100,
                                           update_every=1, # Number of documents to be intereated through for each update, 0 for batch learning, 1 for online iterative learning
                                           chunksize=100, # number of docs in training chunk
                                           passes=10, # total number of passes through the corpus during training
                                           alpha='auto', #hyperparamter that affects sparsity of topics, default = 1.0/num_topics
                                           per_word_topics=True)

lda_model.save('lda_model')

/home/ubuntu/thesis_env2/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
test = LdaModel.load('lda_model')

In [16]:
# Can also create tf-idf
tfidf = gensim.models.TfidfModel(corpus_bow)
corpus_tfidf = tfidf[corpus_bow]
# for doc in corpus_tfidf:
#     pprint(doc)
#     break

In [ ]:
# Build LDA model - tfidf
# Started  still going 
lda_model_tfidf = gensim.models.LdaMulticore(corpus=corpus_tfidf,
                                           id2word=dict_top_mod,
                                           num_topics=500, 
                                           random_state=100,
                                           chunksize=100, # number of docs in training chunk
                                           passes=10, # total number of passes through the corpus during training
                                           per_word_topics=True,
                                           workers=4)

lda_model_tfidf.save('lda_tfidf_model')

Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/queues.py", line 239, in _feed
    obj = _ForkingPickler.dumps(obj)
  File "/usr/lib/python3.8/multiprocessing/reduction.py", line 51, in dumps
    cls(buf, protocol).dump(obj)
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/queues.py", line 239, in _feed
    obj = _ForkingPickler.dumps(obj)
  File "/usr/lib/python3.8/multiprocessing/reduction.py", line 51, in dumps
    cls(buf, protocol).dump(obj)
MemoryError
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/queues.py", line 239, in _feed
    obj = _ForkingPickler.dumps(obj)
  File "/usr/lib/python3.8/multiprocessing/reduction.py", line 51, in dumps
    cls(buf, protocol).dump(obj)
MemoryError
MemoryError
Process ForkPoolWorker-3:
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/process.py", line 313, in _bootstrap
    self.run()
  File "/usr/lib/python3.8/multiprocessing/

In [26]:
# Print the Keyword in the topics
print(len(lda_model.print_topics()))
pprint(lda_model.print_topics())

20
[(127,
  '0.000*"bearded" + 0.000*"redlener" + 0.000*"emigre" + '
  '0.000*"disenfranchise" + 0.000*"cubanamerican" + 0.000*"centerright" + '
  '0.000*"henson" + 0.000*"ucsf" + 0.000*"unbelievably" + 0.000*"dobb"'),
 (86,
  '0.000*"bearded" + 0.000*"redlener" + 0.000*"emigre" + '
  '0.000*"disenfranchise" + 0.000*"cubanamerican" + 0.000*"centerright" + '
  '0.000*"henson" + 0.000*"ucsf" + 0.000*"unbelievably" + 0.000*"dobb"'),
 (247,
  '0.000*"bearded" + 0.000*"redlener" + 0.000*"emigre" + '
  '0.000*"disenfranchise" + 0.000*"cubanamerican" + 0.000*"centerright" + '
  '0.000*"henson" + 0.000*"ucsf" + 0.000*"unbelievably" + 0.000*"dobb"'),
 (8,
  '0.000*"bearded" + 0.000*"redlener" + 0.000*"emigre" + '
  '0.000*"disenfranchise" + 0.000*"cubanamerican" + 0.000*"centerright" + '
  '0.000*"henson" + 0.000*"ucsf" + 0.000*"unbelievably" + 0.000*"dobb"'),
 (93,
  '0.000*"bearded" + 0.000*"redlener" + 0.000*"emigre" + '
  '0.000*"disenfranchise" + 0.000*"cubanamerican" + 0.000*"centerright"

/home/ubuntu/thesis_env2/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [27]:
doc_lda = lda_model[corpus_bow]
doc_lda

/home/ubuntu/thesis_env2/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Compute Model Perplexity and Coherence Score

The model with the lowest perplexity is generally considered the 'best'

Topic Coherence scores topics by measuring the degree of semantic similarity between high scoring words in the topic. These measurements help distinguish between topics that are semantically interpretable topics and topics that are artifacts of statistical inference
* C_v measure is based on a sliding window, one-set segmentation of the top words and an indirect confirmation measure that uses normalized pointwise mutual information (NPMI) and the cosine similarity

In [31]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus_bow))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=dict_top_mod, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -70.69929318668925

Coherence Score:  0.40956980178957886


### Visualize the topics-keywords

Examine the produced topics and the associated keywords. 

Each bubble on the left-hand side plot represents a topic. The larger the bubble, the more prevalent is that topic.
* A good topic model will have fairly big, non-overlapping bubbles scattered throughout the chart instead of being clustered in one quadrant.
* A model with too many topics, will typically have many overlaps, small sized bubbles clustered in one region of the chart

In [34]:
# Visualize the topics
pyLDAvis.enable_notebook()
#vis = pyLDAvis.gensim_models.prepare(lda_model, corpus_bow, dict_top_mod)
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus_bow, dict_top_mod, mds='mmds')
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
39    -0.116425  0.509567       1        1  8.516601
233   -0.517081  0.101589       2        1  5.442988
40     0.509831  0.191219       3        1  4.169913
59     0.514922  0.091478       4        1  3.531670
248    0.542412  0.078122       5        1  2.489091
...         ...       ...     ...      ...       ...
78    -0.016120 -0.017159     246        1  0.000438
222   -0.016120 -0.017159     247        1  0.000437
47    -0.016120 -0.017159     248        1  0.000436
86    -0.016120 -0.017159     249        1  0.000436
127   -0.016120 -0.017159     250        1  0.000436

[250 rows x 5 columns], topic_info=             Term           Freq          Total  Category  logprob  loglift
646          sale  360321.000000  360321.000000   Default  30.0000  30.0000
1125        group  378239.000000  378239.000000   Default  29.0000  29.0000
967         march  387207.000000  387207.000000   Default  28.0000  28.0000
1240      support  321592.000000  321592.000000   Default  27.0000  27.0000
265          yous  346703.000000  346703.000000   Default  26.0000  26.0000
...           ...            ...            ...       ...      ...      ...
25          buyer       0.008112   20223.022548  Topic250 -11.0715  -2.3853
26        capital       0.008112  186328.060639  Topic250 -11.0715  -4.6060
27     capitalize       0.008112    6300.350772  Topic250 -11.0715  -1.2191
28            cat       0.008112   17259.953940  Topic250 -11.0715  -2.2269
29    caterpillar       0.008112    1808.315586  Topic250 -11.0715   0.0291

[11427 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
3202     47  0.995099      10th
2844    216  0.994711      11th
2782     55  0.995529     14day
2845     91  0.992611      1960
2846    201  0.994025      1980
...     ...       ...       ...
5341    155  0.994955      zhao
5043     44  0.990406  zhejiang
4848     44  0.974955     zhong
2226    134  0.997397      zinc
2227     77  0.999805      zoom

[5430 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[40, 234, 41, 60, 249, 218, 24, 3, 169, 82, 164, 57, 109, 31, 70, 213, 190, 194, 17, 49, 208, 83, 158, 243, 112, 6, 130, 2, 227, 182, 54, 108, 99, 59, 58, 183, 177, 162, 93, 122, 84, 153, 231, 18, 149, 16, 85, 12, 241, 150, 47, 76, 46, 39, 166, 139, 138, 98, 66, 55, 23, 42, 148, 235, 184, 135, 181, 36, 229, 117, 180, 222, 145, 189, 27, 250, 121, 127, 141, 133, 81, 119, 197, 246, 64, 219, 10, 174, 230, 33, 8, 96, 172, 61, 228, 201, 7, 67, 151, 233, 110, 240, 146, 78, 124, 123, 136, 74, 131, 206, 192, 43, 77, 13, 69, 111, 220, 102, 125, 34, 88, 156, 209, 132, 217, 97, 168, 205, 163, 167, 207, 71, 11, 51, 1, 126, 140, 104, 176, 157, 105, 193, 45, 115, 73, 224, 114, 165, 80, 120, 113, 15, 211, 100, 170, 35, 32, 204, 95, 90, 144, 202, 116, 103, 199, 210, 244, 14, 161, 215, 63, 232, 147, 187, 225, 65, 238, 86, 21, 171, 137, 101, 20, 44, 106, 245, 50, 22, 154, 72, 195, 107, 129, 19, 191, 29, 239, 5, 92, 160, 175, 53, 173, 91, 242, 142, 178, 38, 226, 52, 30, 214, 68, 159, 118, 236, 4, 134, 75, 203, 37, 198, 56, 26, 89, 186, 216, 185, 200, 188, 62, 9, 25, 179, 247, 196, 155, 152, 248, 221, 94, 237, 143, 28, 212, 79, 223, 48, 87, 128])

### Finding dominant topic per document

In [47]:
print(len(lda_model[corpus_bow]))

365200


In [51]:
lda_model[corpus_bow][0][0]

[(2, 0.023756381),
 (11, 0.04615978),
 (13, 0.0123692015),
 (15, 0.031191511),
 (16, 0.017763438),
 (23, 0.013469117),
 (26, 0.030842746),
 (30, 0.011817191),
 (39, 0.1146293),
 (40, 0.012883494),
 (41, 0.01467653),
 (45, 0.012768744),
 (53, 0.03162221),
 (56, 0.011454442),
 (59, 0.019641891),
 (79, 0.010472117),
 (107, 0.013860089),
 (108, 0.017562041),
 (150, 0.024380412),
 (163, 0.03631914),
 (182, 0.023081202),
 (189, 0.028450739),
 (207, 0.099589355),
 (217, 0.013436001),
 (229, 0.0103148725),
 (233, 0.022963729),
 (242, 0.02481378),
 (248, 0.010170278)]

In [56]:
for item in lda_model.get_document_topics(corpus_bow[0]):
    print(item)

(2, 0.023756288)
(11, 0.046159673)
(13, 0.0123692015)
(15, 0.031192806)
(16, 0.017763734)
(23, 0.01346924)
(26, 0.030843128)
(30, 0.01181722)
(39, 0.11463155)
(40, 0.012883609)
(41, 0.01467653)
(45, 0.012768736)
(53, 0.031622123)
(56, 0.011454735)
(59, 0.01963814)
(79, 0.010472117)
(107, 0.013859884)
(108, 0.017562682)
(150, 0.024380416)
(163, 0.03631969)
(182, 0.023081206)
(189, 0.028451066)
(207, 0.09957951)
(217, 0.013436319)
(229, 0.010314865)
(233, 0.022962425)
(242, 0.024813931)
(248, 0.010170266)


In [ ]:
sorted(row, key=lambda x: (x[1]), reverse=True)

In [60]:
def format_topics_sentences(ldamodel, corpus, texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in tqdm(enumerate(ldamodel[corpus])):
        row = sorted(row, key=lambda x: (x[0]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus_bow, texts=df['content_processed'])

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

365200it [6:48:26, 14.90it/s]


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,248.0,0.0102,"measure, announce, national, allow, remain, cl...",end year corner past time think positioning fo...
1,1,249.0,0.0701,"job, survey, insurance, unemployment, employme...",labor markets closely_watched segment economy ...
2,2,233.0,0.0963,"statement, future, current, operation, impact,...",forgiven think tesla nasdaq tsla little big pi...
3,3,249.0,0.0149,"job, survey, insurance, unemployment, employme...",cios kicked ai item watch competition agenda a...
4,4,245.0,0.1546,"brand, retailer, instagram, collection, fashio...",coronavirus pandemic life returns normal celeb...
5,5,248.0,0.0130,"measure, announce, national, allow, remain, cl...",webinar unpack global trends insights fintech ...
6,6,248.0,0.0141,"measure, announce, national, allow, remain, cl...",calmaine calm reported earnings morning good s...
7,7,248.0,0.0519,"measure, announce, national, allow, remain, cl...",chinese health authorities able identify myste...
8,8,248.0,0.0202,"measure, announce, national, allow, remain, cl...",yous agencies urging proactive measures knowin...
9,9,248.0,0.0142,"measure, announce, national, allow, remain, cl...",peak earnings season kick week reports big ban...


In [61]:
df_dominant_topic.to_pickle('./data/df_lda_dominant_topic_2.pickle')

### Find the most representative document for each topic
Sometimes just the topic keywords may not be enough to make sense of what a topic is about. So, to help with understanding the topic, you can find the documents a given topic has contributed to the most and infer the topic by reading that document.

The tabular output above actually has 20 rows, one each for a topic. It has the topic number, the keywords, and the most representative document. The Perc_Contribution column is nothing but the percentage contribution of the topic in the given document.

In [ ]:
# Group top 5 sentences under each topic
sent_topics_sorted = pd.DataFrame()

sent_topics_out = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorted = pd.concat([sent_topics_sorted, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorted.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorted.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorted.head()

### Topic distribution across documents
Understand the volume and distribution of topics in order to judge how widely it was discussed.

In [35]:
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# Concatenate Column wise
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# Show
df_dominant_topics

NameError: name 'df_topic_sents_keywords' is not defined